In [ ]:
import json
import pandas
import dask.dataframe as dd

In [ ]:
with open("../data/scrapy_data/news.json", "r") as f:
    jsn = json.load(f)

type(jsn)

In [ ]:
from urllib.parse import urlparse


def extract_hostname(url):
    """
    Extract the hostname from a given URL.

    Args:
    url (str): The URL from which the hostname is to be extracted.

    Returns:
    str: The extracted hostname.
    """
    # Parse the URL to extract components
    parsed_url = urlparse(url)

    # Return the hostname
    return parsed_url.hostname

In [ ]:
url = jsn[0]["url"]
hostname = extract_hostname(url)
hostname

In [ ]:
df = pandas.DataFrame.from_dict(jsn)

In [ ]:
df

In [ ]:
# ddf = dd.from_pandas(df, chunksize=4096)

In [ ]:
# ddf.head(3)

In [ ]:
# %%time
# ddf["url"].apply(extract_hostname, meta=("url", "str"))

In [ ]:
# html = ddf.loc[:3].html.compute()[0]
# html

In [ ]:
import re
from bs4 import BeautifulSoup


def extract_dates_from_html(html_content: str):
    # 日付の正規表現パターン
    date_patterns = [
        r"\d{4}年\d{1,2}月\d{1,2}日",  # 例: 2023年4月20日
        r"\d{4}/\d{1,2}/\d{1,2}",  # 例: 2023/04/20
        r"\d{4}\.\d{1,2}\.\d{1,2}",  # 例: 2023.04.20
        r"\d{4}\-\d{1,2}\-\d{1,2}",  # 例: 2023-04-20
        # 他の日付フォーマットがあればここに追加
    ]

    # BeautifulSoupでHTMLを解析
    # soup = BeautifulSoup(html_content, "html.parser")
    soup = BeautifulSoup(html_content, "lxml")
    text = soup.get_text()

    # 日付の抽出
    dates = []
    for pattern in date_patterns:
        matches = re.findall(pattern, text)
        dates.extend(matches)

    return dates

In [ ]:
dates = df.iloc[:5].html.apply(extract_dates_from_html)
dates

In [ ]:
import re


html = df.html.iloc[1]
soup = BeautifulSoup(html, "lxml")

# print(re.sub(r"\n\n+", "\n", soup.get_text()))
df.url[:3]

In [ ]:
import requests


response = requests.get(
    "https://xtech.nikkei.com/atcl/nxt/column/18/00001/08676/?i_cid=nbpnxt_pgmn_topit"
)
html = response.text
soup = BeautifulSoup(html, "lxml")

In [ ]:
soup.find_all("div", attrs={"class": re.compile(r".*date.*")})

In [ ]:
soup.find_all("time", attrs=["datetime"])

In [ ]:
soup.find_all("time", attrs={"class": re.compile(r".*header.*")})

In [ ]:
soup.find("time", attrs={"class": re.compile(r".*header.*")})

In [ ]:
# url = "https://diamond.jp/articles/-/333283"
url = "https://www.itmedia.co.jp/news/"
url = "https://www.itmedia.co.jp/news/articles/2312/01/news113.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, "lxml")
soup.find("time")